Biblioteki

In [1]:
import os
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ReduceLROnPlateau
import shutil
from sklearn.model_selection import train_test_split

Ścieżki i wstępne ustawienia

In [2]:
# Ścieżka do katalogu z obrazami
edge_count = '4' # Liczba wierzchołków do nauki modelu
data_dir_generated = '../graph_generator/' + edge_count

# Wielkość obrazów i wielkość partii
img_height, img_width = 180, 180
batch_size = 32

In [3]:
def rgb_to_grayscale(images):
    return tf.image.rgb_to_grayscale(images)

Przenoszenie do katalogów

In [4]:
# Pobierz listę wszystkich katalogów z grafami
all_images_catalogs = os.listdir(data_dir_generated)

# Stwórz katalog "dane", jeśli jeszcze nie istnieje
data_dir = 'graphs_crossvalid'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
else:
    # Jeśli katalog "graphs_crossvalid" nie jest pusty, wyczyść go
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Skopiuj wszystkie obrazy do katalogu "graphs_crossvalid"
for catalog in all_images_catalogs:
    catalog_path = os.path.join(data_dir_generated, catalog)
    all_images = os.listdir(catalog_path)
    
    # Stwórz podkatalog w katalogu "graphs_crossvalid"
    new_catalog_path = os.path.join(data_dir, catalog)
    if not os.path.exists(new_catalog_path):
        os.makedirs(new_catalog_path)
    
    for image in all_images:
        shutil.copy(os.path.join(catalog_path, image), os.path.join(new_catalog_path, image))
        
# Pobierz listę wszystkich obrazów z katalogu "dane"
all_images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(data_dir) for f in filenames if os.path.splitext(f)[1] == '.png']

In [5]:
# Generowanie danych treningowych
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
class_names = train_ds.class_names
train_ds = train_ds.map(lambda x, y: (rgb_to_grayscale(x), y))

# Generowanie danych walidacyjnych
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
val_ds = val_ds.map(lambda x, y: (rgb_to_grayscale(x), y))

Found 2200 files belonging to 5 classes.
Using 1760 files for training.
Found 2200 files belonging to 5 classes.
Using 440 files for validation.


Model

In [6]:
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
history = []
all_images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(data_dir) for f in filenames if os.path.splitext(f)[1] == '.png']

for train_index, val_index in kfold.split(all_images):
  train_images = [all_images[i] for i in train_index]
  validation_images = [all_images[i] for i in val_index]

  # Dane treningowe
  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      labels='inferred',
      label_mode='int',
      image_size=(img_height, img_width),
      batch_size=batch_size)
  class_names = train_ds.class_names
  train_ds = train_ds.map(lambda x, y: (rgb_to_grayscale(x), y))

  # Dane walidacyjne
  val_ds = tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      labels='inferred',
      label_mode='int',
      image_size=(img_height, img_width),
      batch_size=batch_size)
  val_ds = val_ds.map(lambda x, y: (rgb_to_grayscale(x), y))

  # Model
  model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(class_names))
  ])

  # Kompilacja modelu
  model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
  )

  # Uczenie modelu
  history.append(model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=75
  ))

Found 2200 files belonging to 5 classes.
Found 2200 files belonging to 5 classes.
Epoch 1/75
69/69 ━━━━━━━━━━━━━━━━━━━━ 14s 173ms/step - accuracy: 0.2455 - loss: 2.5972 - val_accuracy: 0.6259 - val_loss: 1.1668
Epoch 2/75
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 156ms/step - accuracy: 0.5853 - loss: 0.9890 - val_accuracy: 0.7932 - val_loss: 0.5502
Epoch 3/75
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 157ms/step - accuracy: 0.7942 - loss: 0.5596 - val_accuracy: 0.8391 - val_loss: 0.5116
Epoch 4/75
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 158ms/step - accuracy: 0.8055 - loss: 0.5043 - val_accuracy: 0.8764 - val_loss: 0.3676
Epoch 5/75
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step - accuracy: 0.8553 - loss: 0.4526 - val_accuracy: 0.4432 - val_loss: 1.3862
Epoch 6/75
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step - accuracy: 0.6101 - loss: 1.0377 - val_accuracy: 0.8495 - val_loss: 0.5590
Epoch 7/75
69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 157ms/step - accuracy: 0.8140 - loss: 0.5687 - val_accuracy: 0.8659 - val_loss: 0.5563
Epoch 8/75
69/6

Średnia dokładność na zbiorze walidacyjnym: 85.00%


In [ ]:
# Tworzymy figurę
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Dla każdego podziału
for i, h in enumerate(history):
    # Wybieramy kolor
    color = plt.cm.viridis(i / len(history))

    # Wykres dokładności
    ax1.plot(h.history['accuracy'], color=color, label='Trening {}'.format(i+1))
    ax1.plot(h.history['val_accuracy'], '--', color=color, label='Walidacja {}'.format(i+1))

    # Wykres straty
    ax2.plot(h.history['loss'], color=color, label='Trening {}'.format(i+1))
    ax2.plot(h.history['val_loss'], '--', color=color, label='Walidacja {}'.format(i+1))

# Ustawiamy tytuły i etykiety
ax1.set_title('Dokładność modelu')
ax1.set_ylabel('Dokładność')
ax1.set_xlabel('Epoka')
ax1.legend()

ax2.set_title('Strata modelu')
ax2.set_ylabel('Strata')
ax2.set_xlabel('Epoka')
ax2.legend()

# Wyświetlamy figurę
plt.show()

# Średnia dokładność na zbiorze walidacyjnym
average_accuracy = np.mean([h.history['val_accuracy'][-1] for h in history])
print("Średnia dokładność na zbiorze walidacyjnym: {:.2f}%".format(average_accuracy * 100))

In [ ]:
h.history

{'accuracy': [0.3137499988079071,
  0.5837500095367432,
  0.8799999952316284,
  0.9624999761581421,
  0.9962499737739563,
  0.9987499713897705,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'loss': [1.3900479078292847,
  0.9927874803543091,
  0.3403078019618988,
  0.11847572028636932,
  0.036509666591882706,
  0.014618010260164738,
  0.006359045393764973,
  0.0029621729627251625,
  0.002751983003690839,
  0.0014443221734836698,
  0.0011847056448459625,
  0.0008710184483788908,
  0.000702761928550899,
  0.0005707400850951672,
  0.000503546092659235,
  0.00

In [ ]:
def get_image_filenames(folder):
    filenames = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            filenames.append(filename)
    return filenames

In [ ]:
# Zmieniamy funkcję, aby przeszukiwała również podkatalogi
def get_image_filenames(directory):
    return [os.path.join(root, name)
            for root, dirs, files in os.walk(directory)
            for name in files
            if name.endswith((".png", ".jpg", ".jpeg"))]

# Ścieżka do obrazu, który chcesz przetestować
test_photo_path = 'test_graphs'

test_photos = get_image_filenames(test_photo_path)
print(len(test_photos))
test_photos

14


['test_graphs\\drawn\\connected-drawn-1.png',
 'test_graphs\\drawn\\cycle-drawn-1.png',
 'test_graphs\\drawn\\full-drawn-1.jpg',
 'test_graphs\\drawn\\full-drawn-2.jpg',
 'test_graphs\\drawn\\full-drawn-3.png',
 'test_graphs\\drawn\\path-drawn-1.png',
 'test_graphs\\drawn\\path-drawn-2.png',
 'test_graphs\\drawn\\path-drawn-3.png',
 'test_graphs\\generated\\cycle-45.png',
 'test_graphs\\generated\\full-113.png',
 'test_graphs\\generated\\path-78.png',
 'test_graphs\\internet\\internet-cycle-1.png',
 'test_graphs\\internet\\internet-full-1.jpg',
 'test_graphs\\internet\\internet-full-2.jpg']

In [ ]:
for test_photo in test_photos:
    # Wybór obrazka testowego
    image_path = test_photo

    # Wczytanie obrazu i dostosowanie go do odpowiedniego rozmiaru
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    
    # Przekształć obraz do skali szarości
    img_array = rgb_to_grayscale(img_array)
    # Rozszerz wymiary obrazu
    img_array = tf.expand_dims(img_array, 0)

    # Przewidywanie klasy obrazu
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "|- {} -| najprawdopodobniej należy do klasy |- {} -| z prawdopodobieństwem {:.2f} procent."
        .format(test_photo, class_names[np.argmax(score)], 100 * np.max(score))
    )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
|- test_graphs\drawn\connected-drawn-1.png -| najprawdopodobniej należy do klasy |- full -| z prawdopodobieństwem 100.00 procent.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
|- test_graphs\drawn\cycle-drawn-1.png -| najprawdopodobniej należy do klasy |- full -| z prawdopodobieństwem 100.00 procent.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
|- test_graphs\drawn\full-drawn-1.jpg -| najprawdopodobniej należy do klasy |- full -| z prawdopodobieństwem 100.00 procent.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
|- test_graphs\drawn\full-drawn-2.jpg -| najprawdopodobniej należy do klasy |- full -| z prawdopodobieństwem 100.00 procent.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
|- test_graphs\drawn\full-drawn-3.png -| najprawdopodobniej należy do klasy |- full -| z prawdopodobieństwem 100.00 procent.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
|- test_graphs\drawn\path-drawn-1.png -| najprawdopodobniej należy do klasy |- full -| z prawdopodobieństwem 100.00 procent.
1/1 ━━━━━━━━━━━